# Method 2: Recursive Character Splitting

This notebook demonstrates **Recursive Character Splitting**, an improvement over fixed-size chunking that attempts to respect some structural boundaries.

## Concept
This method splits text based on a hierarchical list of separator characters. For code, this can be adapted to be language-aware by prioritizing separators like class and function definitions. It recursively works its way down the list of separators until the chunks are a manageable size.

### Pros:
- **Better Semantic Boundaries:** Does a much better job of creating meaningful chunks than the fixed-size version, especially when configured with language-specific delimiters.
- **Good Fallback:** A great strategy when a full AST parser isn't available or fails.

### Cons:
- **Can Be Brittle:** May fail with unusually formatted code and doesn't have the guaranteed understanding of an AST.
- **Requires Customisation:** The separator hierarchy needs to be defined for each programming language.

In [1]:
# We'll use LangChain's implementation for simplicity
%pip install langchain tiktoken

  Using cached langchain-0.3.26-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_text_splitters-0.3.8-py3-none-any.whl.metadata (1.9 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached charset_normalizer-3.4.2-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.wh

## Sample Code for Demonstration
We will use the following block of Python code as the input.

In [2]:
sample_code = """
# Utility functions for string manipulation

def say_hello(name: str):
    \"\"\"A simple function to greet someone.\"\"\"
    print(f"Hello, {name}!")

def say_goodbye(name: str):
    \"\"\"A simple function to say goodbye.\"\"\"
    print(f"Goodbye, {name}!")

class StringHelper:
    \"\"\"A class with advanced string operations.\"\"\"
    def __init__(self, text: str):
        self.text = text

    def reverse(self) -> str:
        # Reverse the text using slicing
        return self.text[::-1]

    def count_vowels(self) -> int:
        vowels = "aeiouAEIOU"
        return sum(1 for char in self.text if char in vowels)

# Standalone script execution
if __name__ == "__main__":
    helper = StringHelper("hello world")
    reversed_text = helper.reverse()
    vowel_count = helper.count_vowels()

    say_hello("Alice")
    print(f"Original: 'hello world', Reversed: '{reversed_text}', Vowels: {vowel_count}")
    say_goodbye("Alice")
"""

## Implementation

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create a splitter specifically for Python using LangChain's presets
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language="python", chunk_size=200, chunk_overlap=30
)

print("--- Method 2: Recursive Character Splitting ---")
recursive_chunks = python_splitter.split_text(sample_code)

for i, chunk in enumerate(recursive_chunks):
    print(f"--- Chunk {i+1} ---")
    print(chunk)

--- Method 2: Recursive Character Splitting ---
--- Chunk 1 ---
# Utility functions for string manipulation

def say_hello(name: str):
    """A simple function to greet someone."""
    print(f"Hello, {name}!")
--- Chunk 2 ---
def say_goodbye(name: str):
    """A simple function to say goodbye."""
    print(f"Goodbye, {name}!")
--- Chunk 3 ---
class StringHelper:
    """A class with advanced string operations."""
    def __init__(self, text: str):
        self.text = text
--- Chunk 4 ---
def reverse(self) -> str:
        # Reverse the text using slicing
        return self.text[::-1]
--- Chunk 5 ---
def count_vowels(self) -> int:
        vowels = "aeiouAEIOU"
        return sum(1 for char in self.text if char in vowels)
--- Chunk 6 ---
# Standalone script execution
if __name__ == "__main__":
    helper = StringHelper("hello world")
    reversed_text = helper.reverse()
    vowel_count = helper.count_vowels()
--- Chunk 7 ---
say_hello("Alice")
    print(f"Original: 'hello world', Reversed